In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import *
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import pdb
import re
import csv
import warnings

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

Ссылки на объявления спарсим с помощью selenium

In [25]:
driver = webdriver.Firefox(
    executable_path=
    r'/home/vlad/anaconda3/lib/python3.7/site-packages/selenium/geckodriver')
#driver=webdriver.Chrome(executable_path=r'/usr/bin/google-chrome')
driver.get('https://auto.ru/')

In [ ]:
show_all_bttn = driver.find_element_by_class_name('IndexMarks__show-all')
show_all_bttn.click()
data_name = driver.find_elements_by_class_name('IndexMarks__item-name')
data_count = driver.find_elements_by_class_name('IndexMarks__item-count')

data_name = list(map(lambda x: x.text, data_name))
data_count = list(map(lambda x: x.text, data_count))

data_dict = {}
for i in range(len(data_name)):
    try:
        data_dict[data_name[i]] = data_count[i]
    except IndexError:
        data_dict[data_name[i]] = 0

car_list_df = pd.DataFrame(pd.Series(data_dict, dtype='int64'))
car_list_df.columns = ['Count']

car_list_df = car_list_df[car_list_df['Count'] > 1000]
car_list_df.to_csv('car_list_df.csv')

На данном этапе можно вручную определить фильтры на сайте auto.ru и развернуть список моделей

In [26]:
def parse_mark_hrefs(mark):
    driver.get(f'https://auto.ru/cars/{mark}/all/')
    input('Выберите необходимые фильтры на сайте')
    models = driver.find_elements_by_class_name(
        'ListingPopularMMM-module__itemName')
    counts = driver.find_elements_by_class_name(
        'ListingPopularMMM-module__itemCount')
    models_dict = {}
    for i in range(len(models)):
        models_dict[models[i].get_attribute('href')] = int(counts[i].text)

    mark_hrefs = []
    for href in models_dict.keys():
        driver.get(href)
        count = 1
        convertible_href = href.split('?')
        for _ in tqdm(range((models_dict[href] // 38) + 1)):
            a_elems = driver.find_elements_by_tag_name('a')
            for a in a_elems:
                if a.get_attribute(
                        'class') == 'Link ListingItemTitle-module__link':
                    mark_hrefs.append(a.get_attribute('href'))
            count += 1
            url = f'{convertible_href[0]}?page={count}&{convertible_href[1]}'
            driver.get(url)

    mark_series = pd.Series(mark_hrefs, name='hrefs')

    mark_series.to_csv(f'{mark}_series.csv')

    return mark_series

Выберем приближенные по классу к BMW марки 

In [3]:
marks = ['mercedes', 'lexus', 'audi', 'bmw','volkswagen']

In [32]:
for mark in marks:
    parse_mark_hrefs(mark)

Выберите необходимые фильтры на сайте


In [4]:
data_marks = []
for i in range(len(marks)):
    mark = pd.read_csv(f'{marks[i]}_series.csv', index_col=0)
    mark.loc[:, 'brand'] = marks[i]
    data_marks.append(mark)
df = pd.concat(data_marks, axis=0)

df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [5]:
df

,hrefs,brand
0,https://auto.ru/cars/used/sale/mercedes/a_klas...,mercedes
1,https://auto.ru/cars/used/sale/mercedes/a_klas...,mercedes
2,https://auto.ru/cars/used/sale/mercedes/a_klas...,mercedes
3,https://auto.ru/cars/used/sale/mercedes/a_klas...,mercedes
4,https://auto.ru/cars/used/sale/mercedes/a_klas...,mercedes
...,...,...
40011,https://auto.ru/cars/used/sale/volkswagen/tran...,volkswagen
40012,https://auto.ru/cars/used/sale/volkswagen/tran...,volkswagen
40013,https://auto.ru/cars/used/sale/volkswagen/tran...,volkswagen
40014,https://auto.ru/cars/used/sale/volkswagen/tran...,volkswagen


Теперь напишем функции для парсинга самих объявлений 

In [6]:
def parse_page(url):
    try:
        base_inf, characters_link = parse_base_inf(url)

        characters_inf, equipment_link = parse_characters_inf(characters_link)

        equipment_inf = parse_equipment_inf(equipment_link)

    except TypeError:
        return dict(), list()
    all_inf = {**base_inf, **characters_inf}

    return all_inf, equipment_inf

response=requests.get('https://auto.ru/cars/used/sale/bmw/5er/1101802277-7ea1e36c/')
response.encoding = 'unicode'
soup=BeautifulSoup(response.text,'html.parser')
soup.find('span',class_='OfferPriceCaption__price').text.replace('\xa0', ' ')

Сбор базовой информации

In [7]:
def parse_base_inf(url):
    response = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0',
    })
    response.encoding = 'unicode'
    if response.status_code != 200:
        return 0
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        base_inf = []
        data = list(
            map(
                lambda x: x.find_all('span'),
                soup.find_all(
                    'li', {
                        'class':
                        lambda x: x and 'CardInfoRow CardInfoRow' in x.split(
                            '_')
                    })))
        for parent in data:
            base_inf.append(
                list(map(lambda x: x.text.replace('\xa0', ' '), parent)))

        catalog_link = soup.find(
            'a',
            class_='Link SpoilerLink CardCatalogLink SpoilerLink_type_default'
        ).get('href')
        base_inf = dict(base_inf)
        base_inf['description'] = soup.find(
            'div', class_='CardDescription__textInner').get_text()
        base_inf['price']=soup.find('span',class_='OfferPriceCaption__price').text.replace('\xa0', ' ')
    except AttributeError:
        return 0
    return base_inf, catalog_link

Сбор информации о характеристиках

In [8]:
def parse_characters_inf(url):
    response = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0',
    })
    if response.status_code != 200:
        return 0
    response.encoding = 'unicode'
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        labels = soup.find_all('dt', class_='list-values__label')
        values = soup.find_all('dd', class_='list-values__value')
        characters_dict = {}
        for i in range(len(labels)):
            characters_dict[labels[i].text] = values[i].text.replace(
                '\u2009', '')
        #pdb.set_trace()
        equipment_link = soup.find_all(
            'a', {
                'class':
                lambda x: x and 'link link_pseudo catalog-tab-item tabs' in x.
                split('__')
            })[-1].get('href')
        characters_dict['name'] = soup.find(
            'h1', class_='catalog__title sidebar2__block').text
    except AttributeError:
        return 0
    return characters_dict, equipment_link

Сбор информации о комплектации

In [9]:
def parse_equipment_inf(url):
    #pdb.set_trace()
    response = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0',
    })
    if response.status_code != 200:
        return 0
    response.encoding = 'unicode'
    equipment_list = []
    try:
        soup = BeautifulSoup(response.text, 'html.parser')
        data = soup.find_all('div', class_='catalog__package-group clearfix')
        for elem in data:
            elem_dict = {}
            elem_dict[elem.find('h3', class_='catalog__h3').text.replace(
                '\u2009', '')] = list(
                    map(lambda x: x.text,
                        elem.find_all('li', class_='catalog__package-list-i')))
            equipment_list.append(elem_dict)
    except AttributeError:
        return 0
    return equipment_list

Создадим csv файл, куда будем записывать результат

In [17]:
test = pd.read_csv('test.csv')
train = pd.DataFrame(columns=list(test.columns) + ['hrefs','price'])
cols = train.columns

In [18]:
train.to_csv('train.csv', index=False)

Структурируем полученную информацию

In [13]:
def from_dict_to_dataframe(row):
    characters, equipment = parse_page(row[0])
    new_row = pd.Series(index=cols)
    new_row['Комплектация'] = str(equipment)
    new_row['brand'] = row[1]
    new_row['hrefs'] = row[0]
    new_row['id'] = np.nan
    try:
        new_row['bodyType'] = characters['Кузов']
    except KeyError:
        new_row['bodyType'] = np.nan
    try:
        new_row['color'] = characters['Цвет']
    except KeyError:
        new_row['color'] = np.nan
    try:
        new_row['fuelType'] = characters['Тип двигателя']
    except KeyError:
        new_row['fuelType'] = np.nan
    try:
        new_row['modelDate'] =  re.findall(pattern=r'.*?\(\d{4} – \d{4}\)',
                                     string=characters['name'])[0]
    except KeyError:
        new_row['modelDate'] = np.nan
    try:
        new_row['name'] =  characters['год выпуска']
    except LookupError:
        new_row['name'] = np.nan
    try:
        new_row['numberOfDoors'] = characters['Количество дверей']
    except KeyError:
        new_row['numberOfDoors'] = np.nan
    try:
        new_row['productionDate'] = re.findall(pattern=r'\d{4}',
                                               string=characters['name'])[0]
    except LookupError:
        new_row['productionDate'] = np.nan
    try:
        new_row['vehicleTransmission'] = characters['Коробка']
    except KeyError:
        new_row['vehicleTransmission'] = np.nan
    try:
        new_row['enginePower'] = characters['Мощность']
    except KeyError:
        new_row['enginePower'] = np.nan
    try:
        new_row['engineDisplacement'] = characters['Объем']
    except KeyError:
        new_row['engineDisplacement'] = np.nan
    try:
        new_row['description'] = characters['description']
    except KeyError:
        new_row['description'] = np.nan
    try:
        new_row['mileage'] = characters['Пробег']
    except KeyError:
        new_row['mileage'] = np.nan
    try:
        new_row['Привод'] = characters['Привод']
    except KeyError:
        new_row['Привод'] = np.nan
    try:
        new_row['Руль'] = characters['Руль']
    except KeyError:
        new_row['Руль'] = np.nan
    try:
        new_row['Состояние'] = characters['Состояние']
    except KeyError:
        new_row['Состояние'] = np.nan
    try:
        new_row['Владельцы'] = characters['Владельцы']
    except KeyError:
        new_row['Владельцы'] = np.nan
    try:
        new_row['ПТС'] = characters['ПТС']
    except KeyError:
        new_row['ПТС'] = np.nan
    try:
        new_row['Таможня'] = characters['Таможня']
    except KeyError:
        new_row['Таможня'] = np.nan
    try:
        new_row['Владение'] = characters['Владение']
    except KeyError:
        new_row['Владение'] = np.nan
    try:
        new_row['price'] = characters['price']
    except KeyError:
        new_row['price']=np.nan
    write_csv(new_row)
    return True

Функция записи в файл

In [14]:
def write_csv(data):
    with open('train.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(data)

Парсинг осуществляем в 12 потоков с отображением шкалы 

In [15]:
def main():
    with Pool(12) as p:
        trash = list(
            tqdm(p.imap(from_dict_to_dataframe, df.values),
                 total=len(df)))

In [19]:
warnings.filterwarnings('ignore')
main()

Смотрим, что получилось

In [20]:
train = pd.read_csv('train.csv')

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40016 entries, 0 to 40015
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              37567 non-null  object 
 1   brand                 40016 non-null  object 
 2   color                 37567 non-null  object 
 3   fuelType              37567 non-null  object 
 4   modelDate             37567 non-null  float64
 5   name                  34525 non-null  object 
 6   numberOfDoors         37567 non-null  float64
 7   productionDate        37567 non-null  float64
 8   vehicleConfiguration  0 non-null      float64
 9   vehicleTransmission   37567 non-null  object 
 10  engineDisplacement    37565 non-null  object 
 11  enginePower           37567 non-null  object 
 12  description           37563 non-null  object 
 13  mileage               37567 non-null  object 
 14  Комплектация          40016 non-null  object 
 15  Привод             